# 목적지향 대화시스템 NLU 시스템

- Text를 입력 받아 OOD Detector를 통해 intent인지 OOD인지 판별
- Intent라면 intent classification을 통해 intent를 예측하고 entity recognition을 통해 slot(정보) 추출
  - 이렇게 추출된 intent와 slot(정보)는 post processing을 통해 NLU Result로 가공됨
- OOD라면 intent classification을 거치지 않고 바로 entity recognition으로 감

In [ ]:
pip install gensim==3.4.0

In [ ]:
pip install sentencepiece

In [ ]:
pip install pytorch-crf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_project_root = 'drive/MyDrive/ColabNotebooks/'

In [ ]:
import os
import sys
import json
import torch
import random

from drive.MyDrive.ColabNotebooks.src.model import BiLSTM_CRF, MakeEmbed, textCNN, DAN, EpochLogger, save
from drive.MyDrive.ColabNotebooks.src.dataset import Preprocessing, MakeDataset

In [ ]:
class NaturalLanguageUnderstanding:
    
    def __init__(self):
        self.dataset = MakeDataset()
        self.embed = MakeEmbed()
        self.embed.load_word2vec()

        self.weights = self.embed.word2vec.wv.vectors
        self.weights = torch.FloatTensor(self.weights)
        
        # setting
        self.intent_clsf = textCNN(self.weights, 256, [3,4,5], 0.5, len(self.dataset.intent_label))
        self.slot_tagger = BiLSTM_CRF(self.weights, self.dataset.entity_label, 256, 128)
        self.ood_detector = DAN(self.weights, 256, 0.5, 2)
        
    # 출력할 NLU의 형태
    # Intent는 한 개만 있음 -> string 형태
    # SLOT은 0개 이상 가능 -> list 형태
    def init_NLU_result(self): # NLU 결과 
        NLU_result = {
                    "INTENT" : "",
                    "SLOT"   :[
            
                        ]
                    }
        return NLU_result
    
    # 학습한 model의 path input
    def model_load(self, intent_path, slot_path, ood_path):
        self.intent_clsf.load_state_dict(torch.load(intent_path))
        self.slot_tagger.load_state_dict(torch.load(slot_path))
        self.ood_detector.load_state_dict(torch.load(ood_path))
        self.intent_clsf.eval()
        self.slot_tagger.eval()
        self.ood_detector.eval()
        
    def predict(self, query):
        x = self.dataset.prep.pad_idx_sequencing(self.embed.query2idx(self.dataset.tokenize(query)))

        x = torch.tensor(x)
        '''
        ood dectector
        '''
        f = self.ood_detector(x.unsqueeze(0))
        ood = torch.argmax(f).tolist()
        print(ood)
        if(ood):
            '''
            intent clsf
            '''
            f = self.intent_clsf(x.unsqueeze(0))

            intent = self.dataset.intents[torch.argmax(f).tolist()]
        else:
            intent = "ood"

        '''
        slot tagger
        '''
        f = self.slot_tagger(x.unsqueeze(0))

        mask = torch.where(x > 0, torch.tensor([1.]), torch.tensor([0.])).type(torch.uint8)

        predict = self.slot_tagger.decode(f,mask.view(1,-1))
        return intent, predict  # intent, slot
    
    # input : intent, 예측된 slot
    def convert_nlu_result(self, query, intent, predict):
        NLU_result = self.init_NLU_result()
        x_token =query.split()

        # slot 태깅된 token 후처리
        '''
        q : 제주도 맛집
        NLU.nlu_predict : ['restaurant', [[12, 0]]]
                                            [12, 0] = [S-LOCATION, O]
                                            
        '''
        slots = [] # 후처리된 SLOT들 저장
        BIE = []   # 후처리가 필요한 SLOT모아두기
        prev = ""; # 이전 slot이 무엇이었는지 저장
        for i, slot in enumerate([self.dataset.entitys[p] for p in predict[0]]):
            name = slot[2:]

            if("S-" in slot):  # 현재 slot이 S로 시작하면
                if(BIE != []):
                    '''
                    B-LOCATION, I-LOCATION, S-DATE 인 경우
                    S-DATE 이전에 BIE에 담긴 B-LOCATION, I-LOCATION 들을 저장

                    ex) 이번 주 제주도 -> BIE에 '이번'과 '주'가 담겨있을 것
                        -> BIE가 비어있지 않으니 slots에 append (각각 list 형태)
                        -> 띄어쓰기로 join 하기
                        -> BIE 비워주기              
                    '''
                    slots.append(prev[2:] +"^"+" ".join(BIE))
                    BIE = []
                # 후처리가 필요한 slot이 없다면 (맨 처음에 S가 등장했거나, O 다음에 S가 등장했으면)
                # slot에 담기
                slots.append(name+"^"+x_token[i])
            elif("B-" in slot):   # 후처리가 완료되지 않았기 때문에 BIE에 append하고 prev 변수를 update
                '''
                뒤에 합쳐야하는 SLOT이 등장할 예정이므로 BIE에 저장
                '''
                BIE.append(x_token[i])
                prev = slot
            # 현재 변수가 I로 시작하면 이전 slot이 B로 시작했는지 확인
            # -> 그랬으면 E가 등장하지 않았기 때문에 (아직 후처리가 완료되지 않아서) BIE에 append
            elif("I-" in slot and "B" in prev):
                '''
                뒤에 합쳐야하는 SLOT이 등장할 예정이므로 BIE에 저장
                '''
                BIE.append(x_token[i])
                prev = slot
            elif("E-" in slot and ("I" in prev or "B" in prev)):
                '''
                SLOT의 끝에 도달했으므로
                BIE에 저장된 TOKEN을 SLOTS으로 JOIN하여 저장
                '''
                BIE.append(x_token[i])
                slots.append(name+"^"+" ".join(BIE))
                BIE = []
            else:
                '''
                O인 경우 BIE가 있으면 저장 
                '''
                if(BIE != []):
                    slots.append(prev[2:]+"^"+" ".join(BIE))
                    BIE = []
        NLU_result["INTENT"] = intent
        NLU_result["SLOT"]   = slots
        return NLU_result
    
    def run(self, query):
        intent, predict = self.predict(query)
        self.nlu_predict = [intent, predict]
        NLU_result = self.convert_nlu_result(query, intent, predict)
        return NLU_result
    
    def print_nlu_result(self, nlu_result):
        print('발화 의도 : ' + nlu_result.get('INTENT'))
        print('발화 개체 : ')
        for slot_concat in nlu_result.get('SLOT'):
            slot_name = slot_concat.split('^')[0]
            slot_value = slot_concat.split('^')[1]
            print("    "+slot_name + " : " + slot_value)

In [ ]:
intent_pretrain_path = drive_project_root+"data/pretraining/save/1_intent_clsf_model/intent_clsf_97.217_steps_33.pt"
entity_pretrain_path = drive_project_root+"data/pretraining/save/1_entity_recog_model/entity_recog_97.192_steps_7.pt"
ood_pretrain_path = drive_project_root+"data/pretraining/save/1_ood_clsf_model/ood_clsf_99.724_steps_5.pt"

In [ ]:
NLU = NaturalLanguageUnderstanding()

NLU.model_load(intent_pretrain_path, entity_pretrain_path, ood_pretrain_path)

In [ ]:
NLU_result = NLU.run("제주도 맛집")

NLU.print_nlu_result(NLU_result)

In [ ]:
NLU_result = NLU.run("오늘 제주도 날씨")

NLU.print_nlu_result(NLU_result)

In [ ]:
NLU_result = NLU.run("제주도")

NLU.print_nlu_result(NLU_result)

In [ ]:
NLU_result = NLU.run("나 내일 제주도 여행가는데 미세먼지 알려줘")

NLU.print_nlu_result(NLU_result)

In [ ]:
NLU_result = NLU.run("나 이번 주 제주도 여행가는데 미세먼지 알려줘")

NLU.print_nlu_result(NLU_result)

In [ ]:
NLU.nlu_predict